<a href="https://colab.research.google.com/github/sakshamhooda/SurakshaNET/blob/main/SurakshaNET_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a test version for model evaluation.
Endpoints are not available here for URL parsing. That is being done manually.

Model used: BARTdistil fine tuned on LORA

Dataset: Open Web Text


In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3", device= 0)

input_text = "Shipping Insurance Remove Insurance"
candidate_labels = ["urgency", "social proof", "scarcity", "misdirection", "forced action", "obstruction", "sneaking", "not dark pattern"]

result = classifier(input_text, candidate_labels)

max_score_label = result["labels"][result["scores"].index(max(result["scores"]))]
print(f"Label : {max_score_label}")

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Label : misdirection


In [ ]:
def classify(text):
  result = classifier(text, candidate_labels)
  max_score_label = result["labels"][result["scores"].index(max(result["scores"]))]
  return max_score_label

In [ ]:
import requests
from bs4 import BeautifulSoup, NavigableString

In [ ]:
def classify_dark_patterns(text, classifier):
    candidate_labels = ["urgency", "social proof", "scarcity", "misdirection", "forced action", "obstruction", "sneaking", "not a dark pattern"]
    result = classifier(text, candidate_labels)
    max_score_label = result["labels"][result["scores"].index(max(result["scores"]))]
    return max_score_label

In [ ]:
def detect_dark_patterns(soup, classifier):
    detected_patterns = []
    for element in soup.find_all(text=True):
        if isinstance(element, NavigableString) and element.strip():
            label = classify_dark_patterns(element, classifier)
            if label != "not a dark pattern":
                detected_patterns.append((str(element), label))
    return detected_patterns

In [ ]:
def get_website_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.text, 'html.parser')
    else:
        print(f"Failed to retrieve website content. Status code: {response.status_code}")
        return None

In [ ]:
url = str(input("URL"))  # Replace with the URL of the website you want to scrape
soup = get_website_content(url)
if soup:
    print("Analyzing website content for dark patterns...")
    detected_patterns = detect_dark_patterns(soup, classifier)

    for element, label in detected_patterns:
        print(f"Detected Element: {element}, Detected Label: {label}")

URLhttps://www.next.co.uk/
Analyzing website content for dark patterns...


<ipython-input-6-6500048ceec4>:3: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for element in soup.find_all(text=True):
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Detected Element: html, Detected Label: misdirection
Detected Element: 
            document.addEventListener("DOMContentLoaded", function (event) {
                // if the content size is more than 900KB, adds data-page-size-above-900KB tag
                const pageSizeThreshhold = 900000;
                const entry = performance.getEntriesByType("navigation")
                    .find(({ name }) => name === location.href);

                var pageSizeLength = entry.decodedBodySize;

                if (pageSizeLength > pageSizeThreshhold) {
                    $("html")[0].setAttribute('data-page-size-above-900KB', 'true');
                    $("html")[0].setAttribute('data-page-size', pageSizeLength);

                    if (dataLayer && Array.isArray(dataLayer)) {
                        dataLayer.push({ 'data-page-size-above-900KB-desktop': 'true' });
                        dataLayer.push({ 'data-page-size': pageSizeLength });
                    }
                }
      